In [1]:
import requests
import pandas as pd
from io import StringIO
import datetime
import os
import talib
import numpy as np
import glob
from sklearn.preprocessing import MinMaxScaler
import warnings
warnings.filterwarnings('ignore')

In [2]:
all_df = []
# 上市股票df
url = "https://isin.twse.com.tw/isin/class_main.jsp?owncode=&stockname=&isincode=&market=1&issuetype=1&industry_code=&Page=1&chklike=Y"
response = requests.get(url)
listed = pd.read_hTR2l(response.text)[0]
listed.columns = listed.iloc[0,:]
listed = listed[["有價證券代號","有價證券名稱","市場別","產業別","公開發行/上市(櫃)/發行日"]]
listed = listed.iloc[1:]
# 上櫃股票df
urlTWO = "https://isin.twse.com.tw/isin/class_main.jsp?owncode=&stockname=&isincode=&market=2&issuetype=&industry_code=&Page=1&chklike=Y"
response = requests.get(urlTWO)
listedTWO = pd.read_hTR2l(response.text)[0]
listedTWO.columns = listedTWO.iloc[0,:]
listedTWO = listedTWO.loc[listedTWO['有價證券別'] == '股票']
listedTWO = listedTWO[["有價證券代號","有價證券名稱","市場別","產業別","公開發行/上市(櫃)/發行日"]]
# 上市股票代號+.TW
stock_1 = listed["有價證券代號"]
stock_num = stock_1.apply(lambda x: str(x) + ".TW")
stock_num.loc[len(stock_num)+1] = '0050.TW'
stock_num.loc[len(stock_num)+1] = '^TWII'
# print(stock_num)
# 上櫃股票代號+.TWO
stock_2 = listedTWO["有價證券代號"]
stock_num2 = stock_2.apply(lambda x: str(x) + ".TWO")
# print(stock_num2)
# concate全部股票代號
stock_num = pd.concat([stock_num, stock_num2], ignore_index=True)
# print(stock_num)
allstock_info = pd.concat([listed, listedTWO], ignore_index=True)
allstock_info.columns = ["ID","有價證券名稱","市場別","產業別","公開發行/上市(櫃)/發行日"]
allstock_info.set_index('ID', inplace = True)
print(allstock_info)
# print(type(allstock_info['ID'].values[0]))


     有價證券名稱 市場別    產業別 公開發行/上市(櫃)/發行日
ID                                   
1101     台泥  上市   水泥工業     1962/02/09
1102     亞泥  上市   水泥工業     1962/06/08
1103     嘉泥  上市   水泥工業     1969/11/14
1104     環泥  上市   水泥工業     1971/02/01
1108     幸福  上市   水泥工業     1990/06/06
...     ...  ..    ...            ...
9949     琉園  上櫃  文化創意業     2003/11/21
9950    萬國通  上櫃   塑膠工業     2004/02/17
9951     皇田  上櫃   電機機械     2003/12/18
9960    邁達康  上櫃   運動休閒     2004/12/06
9962     有益  上櫃   鋼鐵工業     2006/07/10

[1788 rows x 4 columns]


In [ ]:
taiwan0050 = pd.read_csv('C:/Users/User/Desktop/StockInfoHub/Stock_Data_Collector/history_data/0050.TW.csv',
                        index_col='Date', parse_dates=['Date'])
for number, d in enumerate(np.arange(-5005, -35, 7)):
    # 改日期
    while(1):
        day = datetime.datetime.strptime(str(datetime.date.today() + datetime.timedelta(days=float(d))) , '%Y-%m-%d' )
        try:
            t = taiwan0050.loc[day]
            print('day', day)
            break
        except:
            d+=1
    ## 把要看日期的個股資料合併(一天)
    allstock = []
    f = 1
    for i, id in enumerate(stock_num):
        
        try:
            address = 'C:/Users/User/Desktop/StockInfoHub/Stock_Data_Collector/history_data/' + id + '.csv'
            stockdata = pd.DataFrame(pd.read_csv(address, index_col='Date', parse_dates=['Date']).loc[day]).transpose()
            if f == 1:
                allstock = stockdata
                f = 0
            else:
                allstock = pd.concat([stockdata, allstock], ignore_index=True)
            # print(id)
        except Exception as e:
            # print(e)
            pass
    allstock.dropna(inplace=True)
    allstock['ID'] = allstock['ID'].astype(int).astype(str)
    allstock.set_index('ID', inplace = True)
    ## 建立選股條件
    #(list(map(lambda x,y , all([x,y], allstock.loc[id, 'RS EMA250rate']>85, allstock.loc[id, 'RS EMA50rate']>70))))
    for id in allstock.index.values:
        try:
            #
            allstock.loc[id, 'Price>150MA'] = (allstock.loc[id, 'Adj Close']>allstock.loc[id, '150MA'])
            allstock.loc[id, 'Price>200MA'] = (allstock.loc[id, 'Adj Close']>allstock.loc[id, '200MA'])
            allstock.loc[id, '200MA trending up 60d'] = (allstock.loc[id, '200MA ROCP 60MA'] >0)
            allstock.loc[id, 'RS 250rate>80'] = (allstock.loc[id, 'RS 250rate'] > 80)
            # allstock.loc[id, '50MA>150MA'] = (allstock.loc[id, '50MA']>allstock.loc[id, '150MA'])
            # allstock.loc[id, '50MA>200MA'] = (allstock.loc[id, '50MA']>allstock.loc[id, '200MA'])
            # allstock.loc[id, '150MA>200MA'] = (allstock.loc[id, '150MA']>allstock.loc[id, '200MA'])
            # allstock.loc[id, '200MA trending up 20d'] = (allstock.loc[id, '200MA ROCP 20MA'] >0)
            allstock.loc[id, 'year high sort'] = (abs((allstock.loc[id, '250Max']-allstock.loc[id, 'Adj Close'])/allstock.loc[id, '250Max'])<0.25)
            allstock.loc[id, 'year low sort'] = ((allstock.loc[id, 'Adj Close']-allstock.loc[id, '250Min'])/allstock.loc[id, '250Min']>0.25)
            
            #ST5
            allstock.loc[id, 'RS 20rate<15'] = (allstock.loc[id, 'RS 20rate'] < 15)
            allstock.loc[id, 'RS 250rate<45'] = (allstock.loc[id, 'RS 250rate'] < 45)
            allstock.loc[id, 'RS 250rate>25'] = (allstock.loc[id, 'RS 250rate'] > 25)
            allstock.loc[id, 'Volume 50MA>100k'] = (allstock.loc[id, 'Volume 50MA'] > 100*1000)
            #ST6-
            allstock.loc[id, 'RS EMA250rate<20'] = (allstock.loc[id, 'RS EMA250rate'] < 20)
            allstock.loc[id, 'RS EMA20rate<20'] = (allstock.loc[id, 'RS EMA20rate'] < 20)
            allstock.loc[id, 'RS EMA20rate>1'] = (allstock.loc[id, 'RS EMA20rate'] > 1)
            allstock.loc[id, 'Volume 50MA>250k'] = (allstock.loc[id, 'Volume 50MA'] > 250*1000)
            # indicator
            allstock.loc[id, 'ATR250/price'] = (allstock.loc[id, 'ATR250'] / allstock.loc[id, 'Adj Close'])
            allstock.loc[id, 'ATR50/price'] = (allstock.loc[id, 'ATR50'] / allstock.loc[id, 'Adj Close'])
            allstock.loc[id, 'ATR20/price'] = (allstock.loc[id, 'ATR20'] / allstock.loc[id, 'Adj Close'])
            allstock.loc[id, 'ATR250/price<0.03'] = (allstock.loc[id, 'ATR250/price'] < 0.03)
            allstock.loc[id, 'ATR250/price<0.5'] = (allstock.loc[id, 'ATR250/price'] < 0.15)
            allstock.loc[id, 'ATR50/price<0.03'] = (allstock.loc[id, 'ATR50/price'] < 0.03)
            allstock.loc[id, 'ATR20/price<0.03'] = (allstock.loc[id, 'ATR20/price'] < 0.03)            
            #All Template
            allstock.loc[id, 'ST5'] = all(allstock.loc[id, ['RS 20rate<15', 'RS 250rate<45', 'RS 250rate>25', 'Volume 50MA>100k']])
            allstock.loc[id, 'ST6'] = all(allstock.loc[id, ['RS EMA250rate<20', 'RS EMA20rate<20', 'RS EMA20rate>1', 'Volume 50MA>250k']])
        except Exception as e:
            print(e)
            print(id, '失敗')
    ## 篩出本日期符合Template的股票ID
    allstock.dropna(inplace=True)
    ST5_ID = allstock.loc[allstock['ST5']].index.values.astype(str)
    ST6_ID = allstock.loc[allstock['ST6'], ['產業別', 'RS EMA250rate']].sort_values(by='RS EMA250rate', ascending=False).iloc[-50:].index.values.astype(str)
    TR1_ID = np.random.choice(allstock.loc[allstock['Volume 50MA>100k']].index.values.astype(str), size=50, replace=False)
    TR2_ID = np.random.choice(allstock.loc[allstock['Volume 50MA>100k']].index.values.astype(str), size=50, replace=False)
    allstock[['TR1','TR2']] = False
    allstock.loc[TR1_ID, 'TR1'] = True
    allstock.loc[TR2_ID, 'TR2'] = True
    ST5_stock_num = len(ST5_ID)
    ST6_stock_num = len(ST6_ID)
    TR1_stock_num = len(TR1_ID)
    TR2_stock_num = len(TR2_ID)
    print('刪除前  ST5 股票數量 : ', ST5_stock_num, end=' | ')
    print('ST6 股票數量 : ', ST6_stock_num, end=' | ')
    print('TR1 股票數量 : ', TR1_stock_num, end=' | ')
    print('TR2 股票數量 : ', TR2_stock_num, end=' | ')
    all_template_ID = list(set(np.concatenate([ST5_ID,ST6_ID,TR1_ID,TR2_ID])))
    # print(all_template_ID)
    apexstock = allstock.loc[all_template_ID, ['產業別', 'ST5', 'ST6', 'TR1', 'TR2'
                                            , 'RS 250rate', 'RS 50rate', 'RS 20rate', 'RS EMA250rate', 'RS EMA50rate', 'RS EMA20rate'
                                            , 'ATR250/price', 'ATR50/price', 'ATR20/price','RS 250EMA is 10MAX','RS 50EMA is 10MAX','RS 20EMA is 10MAX'
                                            , 'Price>150MA', 'Price>200MA','year high sort', 'year low sort'
                                            , '200MA trending up 60d', 'Volume 50MA>100k', 'Volume 50MA>250k']]
    # 防止用濾網篩選前50個的template混亂
    apexstock[['ST5','ST6','TR1','TR2']] = False
    apexstock.loc[ST5_ID,'ST5'] = True
    apexstock.loc[ST6_ID,'ST6'] = True
    apexstock.loc[TR1_ID,'TR1'] = True
    apexstock.loc[TR2_ID,'TR2'] = True
    apexstock[['RS 250rate', 'RS 50rate', 'RS 20rate', 'RS EMA250rate', 'RS EMA50rate', 'RS EMA20rate']] = apexstock[['RS 250rate', 'RS 50rate', 'RS 20rate', 'RS EMA250rate', 'RS EMA50rate', 'RS EMA20rate']].astype(float).round(1)
    column_name = ['產業別', 'ST5', 'ST6', 'TR1', 'TR2'
                , 'S250rate', 'S50rate', 'S20rate', 'ES250rate', 'ES50rate', 'ES20rate'
                , 'ATR250/price', 'ATR50/price', 'ATR20/price','ES250 is 10D MAX','ES50 is 10D MAX','ES20 is 10D MAX'
                , 'Price>150MA', 'Price>200MA','year high sort', 'year low sort'
                , '200MA trending up 60d', 'Volume 50MA>100k', 'Volume 50MA>250k']

    apexstock.columns = column_name
    apexstock = apexstock.dropna()
    print(f'apexstock shape:{apexstock.shape}')
    
    profit_list = []
    IDs = apexstock.index.values
    day1W = datetime.datetime.strptime(str(datetime.date.today() + datetime.timedelta(days=float(d+7))) , '%Y-%m-%d' )
    day2W = datetime.datetime.strptime(str(datetime.date.today() + datetime.timedelta(days=float(d+14))) , '%Y-%m-%d' )
    day1M = datetime.datetime.strptime(str(datetime.date.today() + datetime.timedelta(days=float(d+31))) , '%Y-%m-%d' )
    day2M = datetime.datetime.strptime(str(datetime.date.today() + datetime.timedelta(days=float(d+62))) , '%Y-%m-%d' ) 
    for i, ID in enumerate(IDs):
        ID = str(int(ID))
        if ID in all_template_ID:
            # print(ID)
            try:
                stock = pd.read_csv(f'C:/Users/User/Desktop/StockInfoHub/Stock_Data_Collector/history_data/{ID}.TW.csv', parse_dates=['Date'], index_col=['Date'])
            except:
                try:
                    stock = pd.read_csv(f'C:/Users/User/Desktop/StockInfoHub/Stock_Data_Collector/history_data/{ID}.TWO.csv', parse_dates=['Date'], index_col=['Date'])
                except Exception as e:
                    print(e)
                    try:
                        apexstock.drop(ID, inplace=True)
                        print(f'drop stock : {ID}')
                    except:
                        pass
                    continue
            got_profit_list = ([ID in ST5_ID, ID in ST6_ID, ID in TR1_ID, ID in TR2_ID])
            # print(got_profit_list)
            try:
                initial_price = stock.loc[stock.index>day, 'Adj Close'].iloc[0]
                oneW_price = stock.loc[stock.index<day1W, 'Adj Close'].iloc[-1]
                twoW_price = stock.loc[stock.index<day2W, 'Adj Close'].iloc[-1]
                oneM_price = stock.loc[stock.index<day1M, 'Adj Close'].iloc[-1]
                twoM_price = stock.loc[stock.index<day2M, 'Adj Close'].iloc[-1]
                # print(f'day1W : {stock.loc[stock.index<day1W, "Adj Close"].index.values[-1]}')
                # print(f'day2W : {stock.loc[stock.index<day2W, "Adj Close"].index.values[-1]}')
                # print(f'day1M : {stock.loc[stock.index<day1M, "Adj Close"].index.values[-1]}')
                # print(f'day2M : {stock.loc[stock.index<day2M, "Adj Close"].index.values[-1]}')
                profit_list.append([ID, got_profit_list[0]*(oneW_price-initial_price)/initial_price,  got_profit_list[0]*(twoW_price-initial_price)/initial_price,  got_profit_list[0]*(oneM_price-initial_price)/initial_price,  got_profit_list[0]*(twoM_price-initial_price)/initial_price
                                    , got_profit_list[1]*(oneW_price-initial_price)/initial_price,  got_profit_list[1]*(twoW_price-initial_price)/initial_price,  got_profit_list[1]*(oneM_price-initial_price)/initial_price,  got_profit_list[1]*(twoM_price-initial_price)/initial_price
                                    , got_profit_list[2]*(oneW_price-initial_price)/initial_price,  got_profit_list[2]*(twoW_price-initial_price)/initial_price,  got_profit_list[2]*(oneM_price-initial_price)/initial_price,  got_profit_list[2]*(twoM_price-initial_price)/initial_price
                                    , got_profit_list[3]*(oneW_price-initial_price)/initial_price,  got_profit_list[3]*(twoW_price-initial_price)/initial_price,  got_profit_list[3]*(oneM_price-initial_price)/initial_price,  got_profit_list[3]*(twoM_price-initial_price)/initial_price])
            except Exception as e:
                print(e)
                try:
                    apexstock.drop(ID, inplace=True)
                    print(f'drop stock : {ID}')
                except:
                    pass
                continue
        else:
            pass
    ST5_stock_num = len(apexstock.loc[apexstock['ST5']].index.values)
    ST6_stock_num = len(apexstock.loc[apexstock['ST6']].index.values)
    TR1_stock_num = len(apexstock.loc[apexstock['TR1']].index.values)
    TR2_stock_num = len(apexstock.loc[apexstock['TR2']].index.values)
    print('刪除後  ST5 股票數量 : ', ST5_stock_num, end=' | ')
    print('ST6 股票數量 : ', ST6_stock_num, end=' | ')
    print('TR1 股票數量 : ', TR1_stock_num, end=' | ')
    print('TR2 股票數量 : ', TR2_stock_num, end=' | ')    
    print(f'apexstock shape : {apexstock.shape}')
    profit_column = ['ST5 1W profit', 'ST5 2W profit', 'ST5 1M profit', 'ST5 2M profit'
                                                   , 'ST6 1W profit', 'ST6 2W profit', 'ST6 1M profit', 'ST6 2M profit'
                                                   , 'TR1 1W profit', 'TR1 2W profit', 'TR1 1M profit', 'TR1 2M profit'
                                                   , 'TR2 1W profit', 'TR2 2W profit', 'TR2 1M profit', 'TR2 2M profit']
    profit_df = pd.DataFrame(profit_list, columns=['ID', 'ST5 1W profit', 'ST5 2W profit', 'ST5 1M profit', 'ST5 2M profit'
                                                   , 'ST6 1W profit', 'ST6 2W profit', 'ST6 1M profit', 'ST6 2M profit'
                                                   , 'TR1 1W profit', 'TR1 2W profit', 'TR1 1M profit', 'TR1 2M profit'
                                                   , 'TR2 1W profit', 'TR2 2W profit', 'TR2 1M profit', 'TR2 2M profit'])
    profit_df.set_index('ID', inplace=True)
    apexstock.loc[apexstock.index.values, profit_column] \
    = profit_df.loc[apexstock.index.values,profit_column]
    apexstock.to_excel('C:/Users/User/Desktop/code/python/stockdata_VCP/VCP_predict_project/ETF test/stock choose for all short-template(1Week)/' + str(day).split(' ')[0] + 'all short-template選股(1Week)' + '.xlsx', encoding='utf-8-sig')
    profit = {'ST5 1W profit':100*profit_df.loc[apexstock.index.values,'ST5 1W profit'].sum()/ST5_stock_num
              ,'ST5 2W profit':100*profit_df.loc[apexstock.index.values,'ST5 2W profit'].sum()/ST5_stock_num
              ,'ST5 1M profit':100*profit_df.loc[apexstock.index.values,'ST5 1M profit'].sum()/ST5_stock_num
              ,'ST5 2M profit':100*profit_df.loc[apexstock.index.values,'ST5 2M profit'].sum()/ST5_stock_num
              ,'ST5 numbers of stock ':ST5_stock_num
              ,'ST6 1W profit':100*profit_df.loc[apexstock.index.values,'ST6 1W profit'].sum()/ST6_stock_num
              ,'ST6 2W profit':100*profit_df.loc[apexstock.index.values,'ST6 2W profit'].sum()/ST6_stock_num
              ,'ST6 1M profit':100*profit_df.loc[apexstock.index.values,'ST6 1M profit'].sum()/ST6_stock_num
              ,'ST6 2M profit':100*profit_df.loc[apexstock.index.values,'ST6 2M profit'].sum()/ST6_stock_num
              ,'ST6 numbers of stock ':ST6_stock_num
              ,'TR1 1W profit':100*profit_df.loc[apexstock.index.values,'TR1 1W profit'].sum()/TR1_stock_num
              ,'TR1 2W profit':100*profit_df.loc[apexstock.index.values,'TR1 2W profit'].sum()/TR1_stock_num
              ,'TR1 1M profit':100*profit_df.loc[apexstock.index.values,'TR1 1M profit'].sum()/TR1_stock_num
              ,'TR1 2M profit':100*profit_df.loc[apexstock.index.values,'TR1 2M profit'].sum()/TR1_stock_num
              ,'TR1 numbers of stock ':TR1_stock_num
              ,'TR2 1W profit':100*profit_df.loc[apexstock.index.values,'TR2 1W profit'].sum()/TR2_stock_num
              ,'TR2 2W profit':100*profit_df.loc[apexstock.index.values,'TR2 2W profit'].sum()/TR2_stock_num
              ,'TR2 1M profit':100*profit_df.loc[apexstock.index.values,'TR2 1M profit'].sum()/TR2_stock_num
              ,'TR2 2M profit':100*profit_df.loc[apexstock.index.values,'TR2 2M profit'].sum()/TR2_stock_num
              ,'TR2 numbers of stock ':TR2_stock_num}
    if number == 0:
        all_df = pd.DataFrame(profit, index = [day])
    else:
        day_profit_df = pd.DataFrame(profit, index = [day])
        all_df = pd.concat([day_profit_df, all_df])
    display(all_df)

In [11]:
all_df.to_excel(r'C:\Users\User\Desktop\code\python\stockdata_VCP\VCP_predict_project\ETF test\all short template (1Week).xlsx')